# GMO Forecasting

*Case: Grantham, Mayo, and Van Otterloo, 2012: Estimating the Equity Risk Premium
[9-211-051].*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from openpyxl import load_workbook
from warnings import filterwarnings
filterwarnings('ignore')

FILE = '../data/gmo_analysis_data.xlsx'

wb = load_workbook(FILE)
print(wb.sheetnames)

info = pd.read_excel(FILE, sheet_name='info')
signals = pd.read_excel(FILE, sheet_name='signals', parse_dates=True, index_col=0)
rf = pd.read_excel(FILE, sheet_name='risk-free rate', parse_dates=True, index_col=0)
total_rets = pd.read_excel(FILE, sheet_name='total returns', parse_dates=True, index_col=0)
info.head()

['info', 'signals', 'risk-free rate', 'total returns']


,Ticker,Description
0,SPX D/P,S&P 500 Index for dividend price ratio
1,SPX E/P,S&P 500 Index for earning price ratio
2,T-Note 10YR,10-year Treasury yield
3,TBill 3M,3-month T-Bill rate
4,SPY,SPY ETF for returns and dividend yield


In [2]:
total_rets.head()

,SPY,GMWAX,GMGEX
date,,,
1996-12-31,-0.023292,-0.022094,-0.013000
1997-01-31,0.061786,0.014735,0.034448
1997-02-28,0.009565,0.022265,0.012733
1997-03-31,-0.045721,-0.015152,-0.016441
1997-04-30,0.064368,-0.006731,0.000000


In [3]:
rf.head() # rate is annualized

,TBill 3M
date,
1996-12-31,0.05171
1997-01-31,0.05147
1997-02-28,0.05220
1997-03-31,0.05322
1997-04-30,0.05233


In [4]:
signals.head()

,SPX D/P,SPX E/P,T-Note 10YR
date,,,
1996-12-31,0.019651,0.051592,0.06418
1997-01-31,0.018455,0.048704,0.06494
1997-02-28,0.018502,0.048434,0.06552
1997-03-31,0.019427,0.055559,0.06903
1997-04-30,0.018430,0.052318,0.06718


## 2 Analyzing GMO

_This section utilizes data in the file `gmo_data.xlsx`._ Convert total returns to **excess returns** using the risk‑free rate.


In [5]:
# Align dataframes to have matching indices and columns before subtraction
rets = total_rets.align(rf, join='inner', axis=0)[0].copy()
for col in total_rets.columns:
    rets[col] = total_rets[col] - rf['TBill 3M'] / 12

# Add the risk-free rate column itself if desired:
rets['TBill 3M'] = rf['TBill 3M'] / 12
total_rets['TBill 3M'] = rf['TBill 3M'] / 12
rets.head()

,SPY,GMWAX,GMGEX,TBill 3M
date,,,,
1996-12-31,-0.027601,-0.026403,-0.017309,0.004309
1997-01-31,0.057497,0.010446,0.030159,0.004289
1997-02-28,0.005215,0.017915,0.008383,0.004350
1997-03-31,-0.050156,-0.019587,-0.020876,0.004435
1997-04-30,0.060008,-0.011092,-0.004361,0.004361


1. **Performance (GMWAX).** Compute **mean**, **volatility**, and **Sharpe ratio** for **GMWAX** over three samples:
   - inception → 2011
   - 2012 → present
   - inception → present  
   Has the mean, vol, and Sharpe changed much since the case?


In [6]:
def maximum_drawdown(returns, relative=False, start_date=None, end_date=None):
    if start_date is not None:
        returns = returns[start_date:]
    if end_date is not None:
        returns = returns[:end_date]
    
    # Convert Series to DataFrame for consistent handling
    if isinstance(returns, pd.Series):
        returns = returns.to_frame()
    
    cum_returns = (1 + returns).cumprod()
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns - rolling_max) / rolling_max
    
    max_drawdown = drawdown.min()
    end_date_idx = drawdown.idxmin()
    
    # Build summary row by row for each column
    summary_rows = []
    for col in drawdown.columns:
        # Get the bottom date for this column
        bottom_date = end_date_idx[col]
        
        # Compute the date of the peak preceding the maximum drawdown
        peak_date = rolling_max.loc[:bottom_date, col].idxmax()
        
        # Find recovery date
        recovery_series = drawdown.loc[bottom_date:, col]
        try:
            recover_date = recovery_series[recovery_series >= 0].index[0]
        except (IndexError, KeyError):
            recover_date = pd.NaT
        
        # Convert dates to date format (drop time)
        try:
            peak_date = pd.to_datetime(peak_date).date()
        except:
            peak_date = pd.NaT
        try:
            bottom_date_formatted = pd.to_datetime(bottom_date).date()
        except:
            bottom_date_formatted = pd.NaT
        try:
            recover_date = pd.to_datetime(recover_date).date()
        except:
            recover_date = pd.NaT
        
        # Compute recovery duration
        if pd.notnull(peak_date) and pd.notnull(recover_date):
            try:
                duration = (pd.Timestamp(recover_date) - pd.Timestamp(peak_date)).days
                duration_str = f"{duration} days"
            except:
                duration_str = pd.NaT
        else:
            duration_str = pd.NaT
        
        # Adjust max drawdown if relative
        mdd_value = max_drawdown[col]
        if relative:
            mdd_value = (mdd_value - returns[col].mean()) / returns[col].std()
        
        summary_rows.append({
            'Max drawdown': mdd_value,
            'Peak': peak_date,
            'Bottom': bottom_date_formatted,
            'Recover': recover_date,
            'Duration (to recover)': duration_str
        })
    
    summary = pd.DataFrame(summary_rows, index=drawdown.columns)
    return summary


def performance_metrics(returns, annualization=1, quantile=.05, relative=False, mdd=True, start_date=None, end_date=None):
    if isinstance(returns, pd.Series):
        metrics = pd.DataFrame(index=[returns.name if returns.name is not None else 0])
    else:
        metrics = pd.DataFrame(index=returns.columns)

    if start_date is not None:
        returns = returns[start_date:]

    if end_date is not None:
        returns = returns[:end_date]

    metrics['Mean'] = returns.mean() * annualization
    metrics['Vol'] = returns.std() * np.sqrt(annualization)
    metrics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(annualization)

    metrics['Min'] = returns.min()
    metrics['Max'] = returns.max()

    metrics['Skewness'] = returns.skew()
    metrics['Kurtosis'] = returns.kurtosis()

    VaR = returns.quantile(quantile)
    CVaR = (returns[returns < returns.quantile(quantile)]).mean()

    if relative:
        VaR = (VaR - returns.mean())/returns.std()
        CVaR = (CVaR - returns.mean())/returns.std()

    metrics[f'VaR ({quantile})'] = VaR
    metrics[f'CVaR ({quantile})'] = CVaR

    if mdd:
        mdd_stats = maximum_drawdown(returns)
        metrics = metrics.join(mdd_stats)

        if relative:
            metrics['Max Drawdown'] = (metrics['Max Drawdown'] - returns.mean())/returns.std()
    return metrics

incep_sample_metrics = performance_metrics(rets, annualization=12, end_date="2011", mdd=False)
sec_sample_metrics = performance_metrics(rets, annualization=12, start_date="2012", mdd=False)
full_sample_metrics = performance_metrics(rets, annualization=12, mdd=False)
incep_sample_max_drawdown_metrics = maximum_drawdown(total_rets, end_date="2011")
sec_sample_max_drawdown_metrics = maximum_drawdown(total_rets, start_date="2012")
full_sample_max_drawdown_metrics = maximum_drawdown(total_rets)

# build hierarchical index according to sample to display metrics for all samples
# Concatenate metrics with a sample indicator column
metrics_list = [
    ('Inception-2011', incep_sample_metrics, incep_sample_max_drawdown_metrics),
    ('2012-Present', sec_sample_metrics, sec_sample_max_drawdown_metrics),
    ('Inception-Present', full_sample_metrics, full_sample_max_drawdown_metrics)
]
# Concatenate metrics and max drawdown DataFrames side by side for each sample, adding a 'Sample' column to each
metrics_df = pd.concat(
    [
        pd.concat([metrics, max_drawdown], axis=1).assign(Sample=name)
        for name, metrics, max_drawdown in metrics_list
    ],
    axis=0
)
metrics_df = metrics_df.reset_index().set_index(['Sample', 'index'])
metrics_df.index.names = ['Sample', 'Fund']
display(metrics_df)

Mean       Vol    Sharpe       Min       Max  \
Sample            Fund                                                         
Inception-2011    SPY       0.035824  0.164163  0.218225 -0.165550  0.109160   
                  GMWAX     0.046422  0.110499  0.420110 -0.149150  0.081877   
                  GMGEX    -0.003823  0.147253 -0.025963 -0.151592  0.096042   
                  TBill 3M  0.028338  0.006038  4.693492 -0.000013  0.005324   
2012-Present      SPY       0.134974  0.139736  0.965917 -0.124693  0.126918   
                  GMWAX     0.049157  0.092661  0.530503 -0.115018  0.074458   
                  GMGEX     0.013182  0.228077  0.057794 -0.658863  0.124668   
                  TBill 3M  0.015678  0.005508  2.846637 -0.000017  0.004552   
Inception-Present SPY       0.083256  0.153416  0.542680 -0.165550  0.126918   
                  GMWAX     0.047730  0.102209  0.466986 -0.149150  0.081877   
                  GMGEX     0.004312  0.189982  0.022695 -0.658863  0.124668   
                  TBill 3M  0.022281  0.006064  3.674409 -0.000017  0.005324   

                            Skewness   Kurtosis  VaR (0.05)  CVaR (0.05)  \
Sample            Fund                                                     
Inception-2011    SPY      -0.514212   0.582770   -0.080228    -0.107769   
                  GMWAX    -0.891709   3.058298   -0.044003    -0.077413   
                  GMGEX    -0.509564   0.672829   -0.082292    -0.100367   
                  TBill 3M  0.006044  -1.573984    0.000038     0.000014   
2012-Present      SPY      -0.504899   0.874237   -0.061134    -0.084464   
                  GMWAX    -0.546601   2.108603   -0.039814    -0.056840   
                  GMGEX    -6.192453  61.007825   -0.065603    -0.151964   
                  TBill 3M  0.956801  -0.623396    0.000004    -0.000002   
Inception-Present SPY      -0.550973   0.776528   -0.078272    -0.098433   
                  GMWAX    -0.778067   2.884218   -0.041147    -0.067310   
                  GMGEX    -5.183655  59.654743   -0.075737    -0.127666   
                  TBill 3M  0.415903  -1.429939    0.000013     0.000002   

                            Max drawdown        Peak      Bottom     Recover  \
Sample            Fund                                                         
Inception-2011    SPY          -0.507976  2007-10-31  2009-02-27         NaT   
                  GMWAX        -0.293614  2007-10-31  2009-02-27  2010-10-29   
                  GMGEX        -0.555630  2007-10-31  2009-02-27         NaT   
                  TBill 3M     -0.000013  2011-09-30  2011-10-31         NaT   
2012-Present      SPY          -0.239280  2021-12-31  2022-09-30  2023-12-29   
                  GMWAX        -0.216795  2021-05-31  2022-09-30  2024-02-29   
                  GMGEX        -0.737364  2014-06-30  2016-11-30         NaT   
                  TBill 3M     -0.000017  2015-07-31  2015-09-30  2015-10-30   
Inception-Present SPY          -0.507976  2007-10-31  2009-02-27  2012-03-30   
                  GMWAX        -0.293614  2007-10-31  2009-02-27  2010-10-29   
                  GMGEX        -0.761812  2007-10-31  2016-11-30         NaT   
                  TBill 3M     -0.000017  2015-07-31  2015-09-30  2015-10-30   

                           Duration (to recover)  
Sample            Fund                            
Inception-2011    SPY                        NaT  
                  GMWAX                1094 days  
                  GMGEX                      NaT  
                  TBill 3M                   NaT  
2012-Present      SPY                   728 days  
                  GMWAX                1004 days  
                  GMGEX                      NaT  
                  TBill 3M               91 days  
Inception-Present SPY                  1612 days  
                  GMWAX                1094 days  
                  GMGEX                      NaT  
                  TBill 3M               91 days

Interestingly, the mean, vol and Sharpe of GWMAX have not changed much between subsamples - they've remained around 4.6-4.9\%, 9-11\%, and .4-.54 respectively.


2. **Tail risk (GMWAX).** For all three samples, analyze extreme scenarios:
   - minimum return
   - 5th percentile (VaR‑5th)
   - maximum drawdown (compute on **total** returns, not excess returns)  
   (a) Does GMWAX have high or low tail‑risk as seen by these stats?  
   (b) Does that vary much across the two subsamples?


(a) GMWAX has lower tail risk than SPY as measured by VaR(0.05), min returns, and max drawdowns. In particular, its VaR is considerably lower than SPY's, and its worst drawdown preceding the GFC (coinciding with SPY's) is -29.3\% in comparison with SPY's halving.

(b) The difference in tail risk statistics between the two samples is mostly in the maximum drawdown. While the second sample had the pandemic shock, the first one had the global financial crisis. As a reult, VaR and min returns aren't that different.


3. **Market exposure (GMWAX).** For all three samples, regress **excess returns of GMWAX** on **excess returns of SPY**:
   - report estimated **alpha**, **beta**, and **R²**
   - is GMWAX a **low‑beta** strategy? has that changed since the case?
   - does GMWAX provide **alpha**? has that changed across subsamples?


In [7]:
from warnings import filterwarnings
filterwarnings('ignore')

# regress GMWAX on SPY
# first sample
def regress_y_on_X(rets, y, X: list[str] | str, start_date = None, end_date = None):
    if isinstance(X, str):
        X = [X]
    X = rets[X]
    y = rets[y]
    if start_date is not None:
        X = X.loc[start_date:]
        y = y.loc[start_date:]
    if end_date is not None:
        X = X.loc[:end_date]
        y = y.loc[:end_date]
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    return results


def print_regression_results(title: str, results):
    print("==========================================")
    print(title)
    print("alpha: {:.6f}".format(results.params[0]))
    print("beta: {:.6f}".format(results.params[1]))
    print("R²: {:.6f}".format(results.rsquared))
    print("==========================================")

def get_regression_results_dataframe(results):
    return pd.DataFrame({
        'alpha': results.params[0],
        'beta': results.params[1],
        'R²': results.rsquared
    }, index=[0])

first_sample_results = regress_y_on_X(rets, 'GMWAX', 'SPY', None, '2011')
first_s_res = (get_regression_results_dataframe(first_sample_results))

second_sample_results = regress_y_on_X(rets, 'GMWAX', 'SPY', '2012', None)
second_s_res = (get_regression_results_dataframe(second_sample_results))

full_sample_results = regress_y_on_X(rets, 'GMWAX', 'SPY', None, None)
full_s_res = (get_regression_results_dataframe(full_sample_results))

# label and concatenate the dataframes
all_res = pd.concat([first_s_res, second_s_res, full_s_res])
all_res.index = ['Inception-2011', '2012-Present', 'Inception-Present']
display(all_res)

,alpha,beta,R²
Inception-2011,0.002250,0.542128,0.648686
2012-Present,-0.002280,0.566914,0.730904
Inception-Present,0.000179,0.547452,0.675236


GMWAX is a middling beta strategy, and that hasn't changed. It is a low-alpha strategy, however, and its alpha has switched signs between samples.

4. **Compare to GMGEX.** Repeat items 1–3 for **GMGEX**. What are key differences between the two strategies?

In [8]:
first_gmgex_res =  get_regression_results_dataframe(regress_y_on_X(rets, 'GMGEX', 'SPY', None, '2011'))
second_gmgex_res = get_regression_results_dataframe(regress_y_on_X(rets, 'GMGEX', 'SPY', '2012', None))
full_gmgex_res = get_regression_results_dataframe(regress_y_on_X(rets, 'GMGEX', 'SPY', None, None))

all_gmgex_res = pd.concat([first_gmgex_res, second_gmgex_res, full_gmgex_res])
all_gmgex_res.index = ['Inception-2011', '2012-Present', 'Inception-Present']
display(all_gmgex_res)

,alpha,beta,R²
Inception-2011,-0.002600,0.764237,0.725898
2012-Present,-0.008139,0.821257,0.253171
Inception-Present,-0.005064,0.781633,0.398403


From the first to the second sample, GMGEX has had significant changes in mean returns (negative to slightly positive), volatility (22\% in later sample versus 14\% in the first), keeping a low absolute Sharpe.

Its tail statistics are worse than SPY or GMWAX across the board (except for its minimum monthly return in the first period being slightly less negative than SPY's worst month). The maximum drawdown and worst days for the second sample seem like measurement error - they are awful. They'd imply an extremely high tail-risk strategy. Finally, it's a much higher beta strategy than GMWAX, though it has lower $R^2$ with SPY over the second sample. Its alpha is consistenly negative as opposed to GMWAX.

## 3 Forecast Regressions

_This section utilizes data in `gmo_data.xlsx`._

1. **Lagged regression.** Consider the regression with predictors lagged one period:

$$
r^{SPY}_{t} \;=\; \alpha^{SPY,X} \;+\; \big(\beta^{SPY,X}\big)^\prime X_{t-1} \;+\; \epsilon^{SPY,X}_{t}
\tag{1}
$$

Estimate (1) and report the **$R^2$**, as well as the OLS estimates for $\alpha$ and $\beta$. Do this for:
   - $X$ as a single regressor, the **dividend–price** ratio ($DP$)
   - $X$ as a single regressor, the **earnings–price** ratio ($EP$)
   - $X$ with **three** regressors: $DP$, $EP$, and the **10‑year yield**  
   For each, report the **$R^2$**.


In [9]:
def lagged_regression(signals, X: list[str] | str, y, lag=1):
    X_lagged = signals[X].shift(lag)
    X_lagged = sm.add_constant(X_lagged)
    
    # Align y and X, drop NaN rows
    combined = pd.concat([y, X_lagged], axis=1).dropna()
    y_clean = combined.iloc[:, 0]
    X_clean = combined.iloc[:, 1:]
    
    model = sm.OLS(y_clean, X_clean)
    results = model.fit()
    return results

# dividend-price ratio lagged regression
dp_results = lagged_regression(signals, signals.columns[0], rets['SPY'])
print("Dividend-price ratio lagged regression:")
print(f"Alpha: {dp_results.params[0]:.6f}")
print(f"Beta: {dp_results.params[1]:.6f}")
print(f"R-squared: {dp_results.rsquared:.6f}")

# earnings-price ratio lagged regression
ep_results = lagged_regression(signals, signals.columns[1], rets['SPY'])
print("Earnings-price ratio lagged regression:")
print(f"Alpha: {ep_results.params[0]:.6f}")
print(f"Beta: {ep_results.params[1]:.6f}")
print(f"R-squared: {ep_results.rsquared:.6f}")

# DP, EP, 10-year yield lagged regression
three_reg_results = lagged_regression(signals, signals.columns[:3], rets['SPY'])
print("DP, EP, 10-year yield lagged regression:")
print(f"Alpha: {three_reg_results.params[0]:.6f}")
print(f"Beta: {three_reg_results.params[1]:.6f}")
print(f"R-squared: {three_reg_results.rsquared:.6f}")


Dividend-price ratio lagged regression:
Alpha: -0.014011
Beta: 1.171755
R-squared: 0.011561
Earnings-price ratio lagged regression:
Alpha: -0.007194
Beta: 0.264020
R-squared: 0.005819
DP, EP, 10-year yield lagged regression:
Alpha: -0.003545
Beta: 0.568923
R-squared: 0.014541


2. **Trading strategy from forecasts.** For each of the three regressions:
   - Build the forecasted SPY return: $\hat r^{SPY}_{t+1}$ (forecast made using $X_t$ to predict $r^{SPY}_{t+1}$).
   - Set the scale (portfolio weight) to $w_t = 100 \,\hat r^{SPY}_{t+1}$.
   - Strategy return: $r^x_{t+1} = w_t\, r^{SPY}_{t+1}$.  
   For each strategy, compute:
   - mean, volatility, Sharpe
   - max drawdown
   - market **alpha**
   - market **beta**
   - market **information ratio**

In [10]:
# build forecasted SPY return
# big issue with this is that it's not separating training and test sets
dp_forecast = dp_results.predict(sm.add_constant(signals[signals.columns[0]]))
ep_forecast = ep_results.predict(sm.add_constant(signals[signals.columns[1]]))
three_reg_forecast = three_reg_results.predict(sm.add_constant(signals))

# compute market information ratio
dp_market_ir = dp_results.params[0] / dp_results.resid.std()
ep_market_ir = ep_results.params[0] / ep_results.resid.std()
three_reg_market_ir = three_reg_results.params[0] / three_reg_results.resid.std()

# set scale
dp_weights = 100 * dp_forecast
ep_weights = 100 * ep_forecast
three_reg_weights = 100 * three_reg_forecast
dp_portfolio_returns = dp_weights * rets['SPY']
ep_portfolio_returns = ep_weights * rets['SPY']
three_reg_portfolio_returns = three_reg_weights * rets['SPY']

# compute strategy returns
dp_metrics = performance_metrics(dp_portfolio_returns, annualization=12, mdd=True)
dp_metrics['Market alpha'] = dp_results.params[0]
dp_metrics['Market beta'] = dp_results.params[1]
dp_metrics['Market information ratio'] = dp_market_ir
dp_metrics.index = ['DP']
ep_metrics = performance_metrics(ep_portfolio_returns, annualization=12, mdd=True)
ep_metrics['Market alpha'] = ep_results.params[0]
ep_metrics['Market beta'] = ep_results.params[1]
ep_metrics['Market information ratio'] = ep_market_ir
ep_metrics.index = ['EP']
three_reg_metrics = performance_metrics(three_reg_portfolio_returns, annualization=12, mdd=True)
three_reg_metrics['Market alpha'] = three_reg_results.params[0]
three_reg_metrics['Market beta'] = three_reg_results.params[1]
three_reg_metrics['Market information ratio'] = three_reg_market_ir
three_reg_metrics.index = ['Three factor regression']
# join 3 dataframes
all_metrics = pd.concat([dp_metrics, ep_metrics, three_reg_metrics])
display(all_metrics[['Mean', 'Vol', 'Sharpe', 'Max drawdown', 'Market alpha', 'Market beta', 'Market information ratio']])


,Mean,Vol,Sharpe,Max drawdown,Market alpha,Market beta,Market information ratio
DP,0.032506,0.166811,0.194868,-0.792464,-0.014011,1.171755,-0.318025
EP,0.040596,0.132153,0.307190,-0.639684,-0.007194,0.264020,-0.162825
Three factor regression,0.047722,0.160610,0.297127,-0.722702,-0.003545,0.568923,-0.080595


3. **Risk characteristics.**
   - For both strategies, the market, and GMO, compute monthly **VaR** at $\pi = 0.05$ (use the historical quantile).
   - The case mentions stocks under‑performed short‑term bonds from 2000–2011. Does the dynamic portfolio above under‑perform the risk‑free rate over this time?
   - Based on the regression estimates, in how many periods do we estimate a **negative risk premium**?
   - Do you believe the dynamic strategy takes on **extra risk**?

In [11]:
display(pd.concat([all_metrics[[ 'VaR (0.05)' ]], full_sample_metrics['VaR (0.05)']]))

,VaR (0.05)
DP,-0.063380
EP,-0.057325
Three factor regression,-0.066701
SPY,-0.078272
GMWAX,-0.041147
GMGEX,-0.075737
TBill 3M,0.000013


In [12]:
dp_metrics = performance_metrics(dp_portfolio_returns, annualization=12, mdd=True, start_date='2000-01-01', end_date='2011-12-31')
ep_metrics = performance_metrics(ep_portfolio_returns, annualization=12, mdd=True, start_date='2000-01-01', end_date='2011-12-31')
three_reg_metrics = performance_metrics(three_reg_portfolio_returns, annualization=12, mdd=True, start_date='2000-01-01', end_date='2011-12-31')
subsample_metrics = pd.concat([dp_metrics, ep_metrics, three_reg_metrics])
subsample_metrics.index = ['DP', 'EP', 'Three factor regression']
print("2000-2011 metrics for dynamic portfolios:")
display(subsample_metrics[['Mean', 'Vol', 'Sharpe', 'Max drawdown']])

2000-2011 metrics for dynamic portfolios:


,Mean,Vol,Sharpe,Max drawdown
DP,-0.019526,0.223490,-0.087370,-0.792464
EP,-0.017559,0.160438,-0.109443,-0.639684
Three factor regression,-0.009227,0.194068,-0.047545,-0.722702


We can see all 3 strategies underperformed the risk-free rate (3-month T-bill rate) over the 2000-2011 period.

The dynamic strategy must take extra risk - it has higher volatility than the earnings-to-price ratio strategy - though it has a higher (less negative Sharpe ratio). The maximum regressed drawdown is still above 100\% - which overstates the risk of a wipeout.

## 4 Out‑of‑Sample Forecasting

_This section utilizes data in `gmo_data.xlsx`._ Focus on using **both** $DP$ and $EP$ as signals in (1). Compute **out‑of‑sample** ($OOS$) statistics:

**Procedure (rolling OOS):**
- Start at $t=60$.
- Estimate (1) using data **through** time $t$.
- Using the estimated parameters and $x_t$, compute the forecast for $t+1$:
  
  $$
  \hat r^{SPY}_{t+1} \;=\; \hat \alpha^{SPY,X}_t \;+\; \big(\hat \beta^{SPY,X}_t\big)^\prime x_t
  $$

- Forecast error: $e^{forecast}_{t+1} = r^{SPY}_{t+1} - \hat r^{SPY}_{t+1}$.
- Move to $t=61$ and iterate.

Also compute the **null** forecast and errors:

$$
\bar r^{SPY}_{t+1} = \frac{1}{t}\sum_{i=1}^t r^{SPY}_i, \qquad
e^{null}_{t+1} = r^{SPY}_{t+1} - \bar r^{SPY}_{t+1}.
$$

1. **Report the out‑of‑sample $R^2$**

$$
R^2_{OOS} \;\equiv\; 1 - \frac{\sum_{i=61}^T \big(e^{forecast}_i\big)^2}{\sum_{i=61}^T \big(e^{null}_i\big)^2}
$$

Did this forecasting strategy produce a positive $R^2_{OOS}$?

In [13]:
# rolling OOS forecasts
t_start = 60

# Prepare data
spy_returns = rets['SPY'].dropna()
signals_dp_ep = signals[signals.columns[:2]].dropna()

# Align
common_idx = spy_returns.index.intersection(signals_dp_ep.index)
spy_returns = spy_returns.loc[common_idx]
signals_dp_ep = signals_dp_ep.loc[common_idx]

forecast_errors = []
null_errors = []
forecasts = []

for t in range(t_start, len(spy_returns) - 1):
    # Training window: from start to time t
    train_returns = spy_returns.iloc[:t+1]
    train_signals = signals_dp_ep.iloc[:t+1]
    
    # Fit lagged regression
    results = lagged_regression(train_signals, train_signals.columns[:2], train_returns, lag=1)
    
    # Forecast t+1 using signal at t
    x_t = sm.add_constant(train_signals.iloc[-1:])[train_signals.columns]  # Last observation (time t)
    forecast = results.predict(x_t)
    forecasts.append(forecast)
    
    # Actual return
    actual = spy_returns.iloc[t+1]
    
    # Errors
    forecast_errors.append(actual - forecast)
    null_errors.append(actual - train_returns.mean())

# OOS R²
R2_OOS = 1 - (np.sum(np.array(forecast_errors)**2) / np.sum(np.array(null_errors)**2))

print(f"Out-of-sample R²: {R2_OOS:.6f}")
if R2_OOS > 0:
    print("Yes, the strategy produced positive OOS R²")
else:
    print("No, the strategy produced negative OOS R² (worse than naive mean forecast)")

ValueError: shapes (1,2) and (3,) not aligned: 2 (dim 1) != 3 (dim 0)

2. **Redo 3.2 with OOS forecasts.** How does the OOS strategy compare to the in‑sample version of 3.2?

3. **Redo 3.3 with OOS forecasts.** Is the point‑in‑time version of the strategy **riskier**?

## 5 EXTRA: ML Forecasts

1. **CART.** Re‑do Section 3 using **CART** (e.g., `RandomForestRegressor` from `sklearn.ensemble`). If you want to visualize, try `sklearn.tree`.
2. **CART, OOS.** Compute out‑of‑sample stats as in Section 4.
3. **Neural Network.** Re‑do Section 3 using a **neural network** (e.g., `MLPRegressor` from `sklearn.neural_network`).
4. **NN & CART, OOS.** Compute out‑of‑sample stats as in Section 4.